In [1]:
! pip install python-igraph
!pip install pyvis
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import re
import csv
import json
import rdflib
import gensim
import pandas as pd
from gensim import corpora
from pymongo import MongoClient
from nltk.corpus import stopwords
import plotly.graph_objects as go
from pyvis.network import Network
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
client = MongoClient("mongodb://localhost:27017/")
db = client['DataTails']
collection = db['Data']  
data_cursor = collection.find({})
DF = pd.DataFrame(list(data_cursor))
print(DF.head())

                        _id  type  \
0  66e965e698330736e0d693d5  None   
1  66e965e698330736e0d693d6  None   
2  66e965e698330736e0d693d7  None   
3  66e965e698330736e0d693d8  None   
4  66e965e698330736e0d693d9  None   

                                           postTitle postDesc  \
0  Adults(especially those over 30), how young do...      NaN   
1  What is a thing that your parents consider nor...      NaN   
2                 What is a smell that comforts you?      NaN   
3  When in history was it the best time to be a w...      NaN   
4    What's the worst way someone broke up with you?      NaN   

              postTime            authorName noOfUpvotes isNSFW  \
0  2024-08-06 01:02:35  Excellent-Studio7257        4068  False   
1  2024-08-06 01:47:22        Bigbumoffhappy        2073  False   
2  2024-08-05 22:21:53         bloomsmittenn        2188  False   
3  2024-08-06 03:32:59   More_food_please_77         778  False   
4  2024-08-05 21:01:39    ImpressiveWrap7363       

In [ ]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in StopWords]
    return ' '.join(tokens)

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl']
DF = DF[Cols]
print(DF.isnull().sum())
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
print(DF.head())
print(DF.isnull().sum())

In [ ]:
GroupedData = DF.groupby('subReddit')
SubReddits = defaultdict(dict)
for subreddit, group in GroupedData:
    print(f"Processing Subreddit: {subreddit}")
    group['Combined'] = group['postTitle'] + " " + group['postDesc']
    group['Tokens'] = group['Combined'].apply(lambda x: word_tokenize(x))
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]
    LDA = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    SubReddits[subreddit]['lda_model'] = LDA
    SubReddits[subreddit]['dictionary'] = dictionary
    SubReddits[subreddit]['corpus'] = corpus
    group['topics'] = [LDA.get_document_topics(bow) for bow in corpus]
    DF.loc[group.index, 'topics'] = group['topics']

In [ ]:
print(DF.shape)
print(DF.describe())
print(DF.head())


In [5]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
custom_stopwords = StopWords | {"make", "thing", "know", "http", "get", "want", "like", "would", "could", "you", "say","also","aita","com","www","made","ago","day","000"}
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove short words
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stopwords and len(word) > 2]
    return tokens

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl','imageUrl','isNSFW']
DF = DF[Cols]
print(DF.isnull().sum())


DF['subReddit'] = DF['subReddit'].fillna('Unknown_SubReddit')
DF['authorName'] = DF['authorName'].fillna('Unknown_Author')
DF['postTitle'] = DF['postTitle'].fillna('Untitled')
DF['postUrl'] = DF['postUrl'].fillna('http://example.com/NOPOST.png')
DF['imageUrl'] = DF['imageUrl'].fillna('http://example.com/NOImage.png')
DF['isNSFW'] = DF['isNSFW'].fillna(False)
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['noOfComments'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
DF['isNSFW'] = DF['isNSFW'].astype(bool)
print(DF.head())
print(DF.isnull().sum())

# Initialize graph and namespaces
g = rdflib.Graph()
SIOC = rdflib.Namespace("http://rdfs.org/sioc/ns#")
DCMI = rdflib.Namespace("http://purl.org/dc/elements/1.1/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
g.bind("sioc", SIOC)
g.bind("dc", DCMI)
g.bind("foaf", FOAF)

# Function to add posts to the graph
def add_post_to_graph(row, index, topic_uri):
    post_uri = rdflib.URIRef(f"http://reddit.com/post{index}")
    subreddit_uri = rdflib.URIRef(f"http://reddit.com/subreddit/{row['subReddit']}")
    author_uri = rdflib.URIRef(f"http://reddit.com/user/{row['authorName']}")
    
    g.add((post_uri, rdflib.RDF.type, SIOC.Post))
    g.add((post_uri, DCMI.title, rdflib.Literal(row['postTitle'])))
    g.add((post_uri, DCMI.description, rdflib.Literal(row['postDesc'])))
    g.add((post_uri, DCMI.date, rdflib.Literal(row['postTime'])))
    g.add((post_uri, SIOC.num_replies, rdflib.Literal(row['noOfUpvotes'])))
    g.add((post_uri, SIOC.link, rdflib.URIRef(row['postUrl'])))
    
    g.add((subreddit_uri, rdflib.RDF.type, SIOC.Container))
    g.add((subreddit_uri, SIOC.has_post, post_uri))
    g.add((post_uri, SIOC.Container, subreddit_uri))

    g.add((author_uri, rdflib.RDF.type, FOAF.Person))
    g.add((author_uri, FOAF.name, rdflib.Literal(row['authorName'])))
    g.add((post_uri, SIOC.has_creator, author_uri))

    comment_uri = rdflib.URIRef(f"http://reddit.com/comment/{index}")
    g.add((comment_uri, rdflib.RDF.type, SIOC.Comment))
    g.add((comment_uri, DCMI.title, rdflib.Literal(row['comments'])))
    g.add((post_uri, SIOC.has_reply, comment_uri))

    g.add((topic_uri, SIOC.has_post, post_uri))

# Group by subreddit and process each one for topic modeling
GroupedData = DF.groupby('subReddit')
all_topics = defaultdict(set)
for subreddit, group in GroupedData:
    print(f"Processing Subreddit: {subreddit}")
    group['Combined'] = group['postTitle'] + group['postDesc']
    
    group['Tokens'] = group['Combined'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in custom_stopwords])
    
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]

    LDA = gensim.models.LdaModel(
        corpus,
        num_topics=5,
        id2word=dictionary,
        passes=15,
        iterations=100,
        random_state=42
    )
    
    for idx, topic in LDA.print_topics(num_words=5):
        topic_words = topic.split(" + ")
        unique_words = {word.split("*")[1].strip('"') for word in topic_words}
        all_topics[subreddit].update(unique_words)

# Save RDF graph in Turtle and JSON-LD formats
g.serialize('D:/FYP/Github/data-tails/Backend/Ontologies/KG1.ttl', format='turtle')
json_ld = g.serialize(format='json-ld', indent=4)
with open("D:/FYP/Github/data-tails/Backend/Ontologies/KG1.json", "w", encoding="utf-8") as f:
    f.write(json_ld)


subReddit           2
postTitle           0
postDesc        15607
postTime            0
authorName        587
noOfUpvotes         0
comments            0
noOfComments        2
postUrl             2
imageUrl            3
isNSFW              1
dtype: int64
   subReddit                                          postTitle postDesc  \
0  AskReddit                   [adult, especially, young, seem]       []   
1  AskReddit  [parent, consider, normal, consider, normal, a...       []   
2  AskReddit                                   [smell, comfort]       []   
3  AskReddit                       [history, best, time, woman]       []   
4  AskReddit                       [worst, way, someone, broke]       []   

             postTime            authorName noOfUpvotes  \
0 2024-08-06 01:02:35  Excellent-Studio7257        4068   
1 2024-08-06 01:47:22        Bigbumoffhappy        2073   
2 2024-08-05 22:21:53         bloomsmittenn        2188   
3 2024-08-06 03:32:59   More_food_please_77         

In [6]:
# Save subreddits and topics to Topics.csv
with open("D:/FYP/Github/data-tails/Backend/Ontologies/Topics.csv", mode='w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["subreddit", "topics"])
    for subreddit, topics in all_topics.items():
        writer.writerow([subreddit, ", ".join(sorted(topics))])

print("Subreddits and topics have been saved to Topics.csv.")



# List of subreddits to include in the new CSV
selected_subreddits = [
    "todayilearned", "news", "mildlyinteresting", "AskReddit", "Damnthatsinteresting", 
    "showerthoughts", "wallstreetbets", "NatureIsFuckingLit", "LifeProTips", "space", 
    "MovieDetails", "tifu", "futurology", "Music", "europe", "unpopularopinion", 
    "pakistan", "islamabad", "lahore", "karachi"
]

# Read the existing topics CSV
topics_df = pd.read_csv("D:/FYP/Github/data-tails/Backend/Ontologies/Topics.csv")

# Filter rows where 'subreddit' is in the selected subreddits list
filtered_topics_df = topics_df[topics_df['subreddit'].isin(selected_subreddits)]

# Save the filtered data to a new CSV file
filtered_topics_csv_path = "D:/FYP/Github/data-tails/Backend/Ontologies/FTopics.csv"
filtered_topics_df.to_csv(filtered_topics_csv_path, index=False)

print(f"Filtered topics saved to {filtered_topics_csv_path}")

Subreddits and topics have been saved to Topics.csv.
Filtered topics saved to D:/FYP/Github/data-tails/Backend/Ontologies/FTopics.csv


### Creating GRAPH STRUCTURE

In [ ]:
import rdflib
from rdflib import URIRef, Literal
from collections import defaultdict
import gensim
from gensim import corpora
import json

# RDF graph initialization
g = rdflib.Graph()
SIOC = rdflib.Namespace("http://rdfs.org/sioc/ns#")
DCMI = rdflib.Namespace("http://purl.org/dc/elements/1.1/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
REDDIT = rdflib.Namespace("http://reddit.com/ns#")  # Custom namespace for Reddit-specific relationships
g.bind("sioc", SIOC)
g.bind("dc", DCMI)
g.bind("foaf", FOAF)
g.bind("reddit", REDDIT)


# Function to add post data to RDF graph
def add_post_to_graph(row, index, topic_uri, subreddit_type_uri, author_type_uri, post_type_uri, comment_type_uri):
    post_uri = URIRef(f"http://reddit.com/post{index}")
    subreddit_uri = URIRef(f"http://reddit.com/subreddit/{row['subReddit']}")
    author_uri = URIRef(f"http://reddit.com/user/{row['authorName']}")
    
    # Add post properties and link to topic
    g.add((post_uri, rdflib.RDF.type, SIOC.Post))
    g.add((post_uri, DCMI.title, Literal(' '.join(row['postTitle']))))
    g.add((post_uri, DCMI.description, Literal(' '.join(row['postDesc']))))
    g.add((post_uri, DCMI.date, Literal(row['postTime'])))
    g.add((post_uri, SIOC.num_replies, Literal(row['noOfUpvotes'])))
    g.add((post_uri, SIOC.link, URIRef(row['postUrl'])))
    g.add((post_uri, SIOC.NSFW, Literal(row['isNSFW'])))
    g.add((post_uri, SIOC.has_type, post_type_uri))
    g.add((post_uri, SIOC.topic, topic_uri))

    # Create the relationships based on the list of relationship types
    # CreatedBy: Indicates that a post is created by an author.
    g.add((post_uri, REDDIT.CreatedBy, author_uri))

    # BelongsTo: Links a post to a subreddit.
    g.add((post_uri, REDDIT.BelongsTo, subreddit_uri))

    # PartOfPost: Indicates that a comment is part of a post (the comment belongs to the post).
    comment_uri = URIRef(f"http://reddit.com/comment/{index}")
    g.add((comment_uri, REDDIT.PartOfPost, post_uri))

    # HasType: Links an entity (post, comment) to its specific type (text post, link post, etc.).
    g.add((post_uri, REDDIT.HasType, post_type_uri))
    g.add((comment_uri, REDDIT.HasType, comment_type_uri))

    # TaggedIn: Indicates that a post or comment is tagged with a specific topic or keyword.
    for tag in row.get('tags', []):  # Assuming tags are available as a list
        tag_uri = URIRef(f"http://reddit.com/tag/{tag}")
        g.add((post_uri, REDDIT.TaggedIn, tag_uri))
        g.add((comment_uri, REDDIT.TaggedIn, tag_uri))

    # UpvotedBy: Indicates that a user has upvoted a post or comment.
    if row.get('upvotedBy'):
        for upvoter in row['upvotedBy']:
            upvoter_uri = URIRef(f"http://reddit.com/user/{upvoter}")
            g.add((post_uri, REDDIT.UpvotedBy, upvoter_uri))
            g.add((comment_uri, REDDIT.UpvotedBy, upvoter_uri))

    # DownvotedBy: Indicates that a user has downvoted a post or comment.
    if row.get('downvotedBy'):
        for downvoter in row['downvotedBy']:
            downvoter_uri = URIRef(f"http://reddit.com/user/{downvoter}")
            g.add((post_uri, REDDIT.DownvotedBy, downvoter_uri))
            g.add((comment_uri, REDDIT.DownvotedBy, downvoter_uri))

    # Moderates: Indicates that a user is a moderator of a subreddit.
    if row.get('moderators'):
        for moderator in row['moderators']:
            moderator_uri = URIRef(f"http://reddit.com/user/{moderator}")
            g.add((subreddit_uri, REDDIT.Moderates, moderator_uri))

    # HasFlair: Links a post or comment to a specific flair (e.g., tags like "Important", "Question", etc.).
    if row.get('flair'):
        flair_uri = URIRef(f"http://reddit.com/flair/{row['flair']}")
        g.add((post_uri, REDDIT.HasFlair, flair_uri))
        g.add((comment_uri, REDDIT.HasFlair, flair_uri))

    # Link post to subreddit and type
    g.add((subreddit_uri, rdflib.RDF.type, SIOC.Container))
    g.add((subreddit_uri, SIOC.has_post, post_uri))
    g.add((subreddit_uri, SIOC.has_type, subreddit_type_uri))
    g.add((post_uri, SIOC.Container, subreddit_uri))

    # Link to author and author type
    g.add((author_uri, rdflib.RDF.type, FOAF.Person))
    g.add((author_uri, FOAF.name, Literal(row['authorName'])))
    g.add((post_uri, SIOC.has_creator, author_uri))
    g.add((author_uri, SIOC.has_type, author_type_uri))

    # Add comments and link to comment type
    g.add((comment_uri, rdflib.RDF.type, SIOC.Comment))
    g.add((comment_uri, DCMI.title, Literal(row['comments'])))
    g.add((post_uri, SIOC.has_reply, comment_uri))
    g.add((comment_uri, SIOC.has_type, comment_type_uri))

# Process each subreddit and create LDA models
GroupedData = DF.groupby('subReddit')
all_topics = defaultdict(set)

for subreddit, group in GroupedData:
    group['Combined'] = group['postTitle'] + group['postDesc']
    group['Tokens'] = group['Combined'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in custom_stopwords])

    # Create dictionary and corpus
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]
    LDA = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15, iterations=100, random_state=42)
    
    for idx, topic in LDA.print_topics(num_words=5):
        topic_uri = URIRef(f"http://reddit.com/topic/{subreddit}_{idx}")
        topic_words = topic.split(" + ")
        unique_words = {word.split("*")[1].strip('"') for word in topic_words}
        all_topics[subreddit].update(unique_words)

        # Define URIs for types
        subreddit_type_uri = URIRef(f"http://reddit.com/type/subreddit/{subreddit}")
        post_type_uri = URIRef(f"http://reddit.com/type/post/{subreddit}")
        comment_type_uri = URIRef(f"http://reddit.com/type/comment/{subreddit}")
        author_type_uri = URIRef(f"http://reddit.com/type/author/{subreddit}")

        # Link subreddit to topic
        subreddit_uri = URIRef(f"http://reddit.com/subreddit/{subreddit}")
        g.add((subreddit_uri, SIOC.has_topic, topic_uri))
        g.add((topic_uri, rdflib.RDF.type, SIOC.Topic))

        # Iterate through each row in the group
        for index, row in group.iterrows():
            add_post_to_graph(row, index, topic_uri, subreddit_type_uri, author_type_uri, post_type_uri, comment_type_uri)

# Save graph
g.serialize('D:/FYP/Github/data-tails/Backend/Ontologies/KG.ttl', format='turtle')
json_ld = g.serialize(format='json-ld', indent=4)
with open("D:/FYP/Github/data-tails/Backend/Ontologies/KG.json", "w", encoding="utf-8") as f:
    f.write(json_ld)

## GraphRAG

In [ ]:
import rdflib
from rdflib import Graph, Namespace, Literal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import subprocess
import re
from isodate import parse_datetime, ISO8601Error

# Namespaces
SIOC = Namespace("http://rdfs.org/sioc/ns#")
DCMI = Namespace("http://purl.org/dc/elements/1.1/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
reddit = Namespace("http://reddit.com/")

# Preprocessing
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text.lower())
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]

def clean_dates(graph):
    for s, p, o in graph.triples((None, DCMI.date, None)):
        if isinstance(o, Literal):
            try:
                parse_datetime(str(o))  # Validate
            except ISO8601Error:
                graph.remove((s, p, o))
                graph.add((s, p, Literal("1970-01-01T00:00:00", datatype=SIOC.date)))

def retrieve_expanded_context(graph, user_query, depth=2):
    sparql_query = f"""
    SELECT ?subject ?predicate ?object
    WHERE {{
        ?subject ?predicate ?object .
        OPTIONAL {{ ?subject <{DCMI.title}> ?title }}
        OPTIONAL {{ ?subject <{DCMI.description}> ?description }}
        FILTER(
            CONTAINS(LCASE(STR(?title)), LCASE("{user_query}")) || 
            CONTAINS(LCASE(STR(?description)), LCASE("{user_query}"))
        )
    }}
    LIMIT {depth * 10}
    """
    results = graph.query(sparql_query)
    return "\n".join([f"{s} --[{p}]--> {o}" for s, p, o in results])

def query_wizardlm2(context, user_query):
    prompt = f"Context:\n{context}\n\nQuestion:\n{user_query}\n"
    try:
        result = subprocess.run(
            ["ollama", "run", "wizardlm2"],
            input=prompt.encode("utf-8"),  # Encode input
            capture_output=True
        )
        if result.returncode != 0:
            raise Exception(result.stderr.decode("utf-8"))
        return result.stdout.decode("utf-8").strip()
    except Exception as e:
        return f"Error querying WizardLM2: {str(e)}"

if __name__ == "__main__":
    kg_file = "./KG.ttl"
    kg = Graph()
    kg.parse(kg_file, format="turtle")
    clean_dates(kg)

    user_query = "What are the causes of OCD?"
    context = retrieve_expanded_context(kg, user_query, depth=3)
    print(f"Retrieved Context:\n{context}")

    response = query_wizardlm2(context, user_query)
    print(f"\nLLM Response:\n{response}")



🔍 Loading Knowledge Graphs...
✅ Loaded KG.json with 63763 entities.


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x0000025BC9088820>
Traceback (most recent call last):
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\rdflib\term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\isodate\isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
  File "c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\isodate\isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: 'Na'


✅ Loaded KG.ttl with 648909 triples.

🔍 Retrieving Relevant Comments...

🤖 Querying Groq...

💡 Groq Response: OCD (Obsessive-Compulsive Disorder) is a complex condition that is not yet fully understood, but research has shed light on some of the possible causes and contributing factors. Here are some of the key theories:

1. **Genetics**: OCD tends to run in families, suggesting a possible genetic component. Researchers have identified several genes that may contribute to an increased risk of developing OCD.
2. **Neurochemistry**: OCD is often linked to imbalances in neurotransmitters, such as serotonin, dopamine, and glutamate, which play a crucial role in brain function and behavior. Abnormalities in these chemicals may contribute to the development of OCD symptoms.
3. **Brain structure and function**: Studies have found differences in brain structure and function in individuals with OCD, including altered activity in areas responsible for emotional regulation, decision-making, and i

### Visualization Recommedation System

In [13]:
import numpy as np
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load NLP models
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define visualization categories conceptually
chart_types = {
    "bar_chart": "Used for comparing categories or ranking values.",
    "line_chart": "Best for showing trends over time or sequential data.",
    "area_chart": "Shows cumulative data trends with a filled area.",
    "scatterplot": "Visualizes relationships or correlations between two variables.",
    "density_facet": "Represents distribution patterns across multiple facets.",
    "gradient_encoding": "Indicates intensity or magnitude with color gradients.",
    "candlestick_chart": "Commonly used for stock market trends and price movements.",
    "stacked_normalized_area_chart": "Shows part-to-whole relationships over time.",
    "circle_packing": "Represents hierarchical relationships in a compact form.",
    "dendrogram": "Illustrates hierarchical clustering or tree structures.",
    "DAG": "Shows directed relationships, commonly used for processes or networks.",
    "treemap": "Depicts hierarchical structures using nested rectangles.",
    "chord_diagram": "Best for visualizing relationships and interactions.",
    "heatmap": "Displays intensity values in a matrix format.",
    "connection_map": "Visualizes spatial relationships and geographic data.",
    "maps": "Represents geographic data with location-based insights.",
    "map_small_multiples": "Shows multiple maps for comparison.",
    "hexbin_map": "Aggregates spatial data into hexagonal bins.",
    "centerline_labelling": "Enhances map readability with labeled centerlines.",
    "voronoi_map": "Divides spatial regions based on distance.",
    "sorted_heatmap": "Ranks and sorts data in a matrix format."
}

# Precompute chart category embeddings
category_embeddings = {
    chart: model.encode(description, normalize_embeddings=True)
    for chart, description in chart_types.items()
}

# Function to extract features from both the query and response
def extract_features(query, response):
    """Extracts numerical data, locations, trends, and relationships from text."""
    combined_text = query + " " + response
    doc = nlp(combined_text)

    numbers = [token.text for token in doc if token.like_num]
    locations = [ent.text for ent in doc.ents if ent.label_ in {"GPE", "LOC"}]

    trend_keywords = {"increase", "decline", "growth", "rise", "drop", "trend", "fall", "reduce", "expansion"}
    trends = [token.lemma_ for token in doc if token.lemma_ in trend_keywords]

    relationship_keywords = {"prefer", "dominate", "compared", "versus", "majority", "minority"}
    relationships = [token.lemma_ for token in doc if token.lemma_ in relationship_keywords]

    ranking_keywords = {"ranking", "top", "best", "percent", "favorite", "most popular"}
    rankings = [token.lemma_ for token in doc if token.lemma_ in ranking_keywords]

    geo_terms = {"earthquake", "magnitude", "epicenter", "seismic", "hurricane", "flood", "storm"}
    geo_features = [token.lemma_ for token in doc if token.lemma_ in geo_terms]

    keywords = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    return {
        "numbers": numbers,
        "locations": locations,
        "trends": trends,
        "relationships": relationships,
        "geo_features": geo_features,
        "rankings": rankings,
        "keywords": keywords
    }

# Adjust similarity scores based on extracted features
def boost_scores(scores, response_type, features):
    """Dynamically boosts scores based on response type and detected keywords."""
    boosts = {
        "numeric": {"bar_chart": 0.6, "line_chart": 0.5, "heatmap": 0.4},
        "geographic": {"maps": 0.7, "connection_map": 0.6, "hexbin_map": 0.5, "voronoi_map": 0.5},
        "textual": {"scatterplot": 0.2, "DAG": 0.2, "chord_diagram": 0.2}
    }

    if response_type in boosts:
        for chart, boost in boosts[response_type].items():
            scores[chart] += boost

    if features["trends"]:
        scores["line_chart"] += 0.6
        scores["area_chart"] += 0.5

    if features["relationships"]:
        scores["bar_chart"] += 0.6
        scores["chord_diagram"] += 0.5

    if features["geo_features"]:
        scores["maps"] += 0.7
        scores["connection_map"] += 0.6
        scores["hexbin_map"] += 0.5

    if features["rankings"]:
        scores["bar_chart"] += 0.7
        scores["sorted_heatmap"] += 0.5

    return scores

# Function to recommend visualizations
def recommend_visualizations(query, response):
    """Recommends the most relevant visualizations based on query and response context."""
    features = extract_features(query, response)
    response_embedding = model.encode(query + " " + response, normalize_embeddings=True)

    similarity_scores = {
        chart: cosine_similarity([response_embedding], [embedding]).flatten()[0]
        for chart, embedding in category_embeddings.items()
    }

    similarity_scores = boost_scores(similarity_scores, "numeric", features)

    top_recommendations = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)[:3]

    return [(chart, round(score, 2)) for chart, score in top_recommendations]


# Example Usage
queries_responses = [
    # r/AskReddit - Opinion-based and personal experiences
    ("What are the most common fears among people?", 
     "A recent poll on r/AskReddit found that 42% of users fear failure, 35% fear public speaking, and 23% fear being alone. Spiders, heights, and drowning were also frequently mentioned."),

    ("What are Reddit users’ favorite fast-food chains?", 
     "Based on a survey in r/AskReddit, 48% of users ranked Chick-fil-A as their favorite, 30% preferred McDonald's, and 22% chose Taco Bell. Reasons included food quality, affordability, and speed."),

    # r/worldnews - Global events & breaking news
    ("What are the latest updates on the Ukraine-Russia conflict?", 
     "Recent reports from r/worldnews indicate increased military activity along the eastern border. A UN spokesperson confirmed ongoing peace talks, but tensions remain high."),

    ("Has there been a major natural disaster recently?", 
     "According to r/worldnews, a Category 4 hurricane made landfall in Florida, causing severe flooding and power outages for over 1 million residents. Relief efforts are underway."),

    # r/technology - Trends in AI, software, and hardware
    ("What are the latest advancements in artificial intelligence?", 
     "A new AI model, GPT-5, was announced in r/technology. It boasts improved reasoning, coding capabilities, and multi-modal integration, making it one of the most advanced LLMs to date."),

    ("What’s the most powerful consumer GPU available?", 
     "A benchmark test shared in r/technology shows that the NVIDIA RTX 4090 outperforms all competitors, with a 15% lead over the AMD Radeon RX 7900 XTX in 4K gaming."),

    # r/science - Research & discoveries
    ("What new medical breakthroughs have been announced?", 
     "Researchers in r/science have developed a new cancer treatment that increases survival rates by 40%. The therapy targets aggressive tumor cells while minimizing side effects."),

    ("Has any new planet been discovered?", 
     "A post in r/science confirms that astronomers using the James Webb Telescope have discovered an Earth-like exoplanet with possible signs of liquid water."),

    # r/politics - Government policies & elections
    ("How are global leaders responding to climate change?", 
     "In r/politics, reports indicate that several world leaders at the UN Climate Summit pledged to reduce carbon emissions by 30% by 2030, with mixed reactions from environmental groups."),

    ("What are the latest updates on the U.S. elections?", 
     "Recent discussions in r/politics suggest that the frontrunners in the U.S. election are gaining traction among young voters, with debates heating up over key policy issues."),

    # r/gaming - Video game trends & preferences
    ("What are the most-played video games in 2024?", 
     "According to a poll in r/gaming, the top 3 most-played games are 'Elden Ring DLC,' 'Baldur’s Gate 3,' and 'Call of Duty: Modern Warfare III.' Online multiplayer games remain dominant."),

    ("Are microtransactions still a problem in gaming?", 
     "A heated debate on r/gaming revealed that 65% of players feel microtransactions ruin the gaming experience, while 35% believe they are acceptable in free-to-play models."),

    # r/space - Astronomy & exploration
    ("What’s the next big space mission?", 
     "According to r/space, NASA's Artemis II mission is set to send astronauts around the Moon next year, marking a significant step toward future Mars exploration."),

    ("Has there been any recent asteroid impact?", 
     "Scientists in r/space confirmed that a small asteroid entered Earth’s atmosphere over the Pacific Ocean, burning up harmlessly before reaching the surface."),
]


for query, response in queries_responses:
    recommended_charts = recommend_visualizations(query, response)
    print(f"\n💬 Query: {query}")
    print(f"💬 Response: {response}")
    print(f"✅ Top 3 Recommended Visualizations: {recommended_charts}")


c:\Users\97156\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



💬 Query: What are the most common fears among people?
💬 Response: A recent poll on r/AskReddit found that 42% of users fear failure, 35% fear public speaking, and 23% fear being alone. Spiders, heights, and drowning were also frequently mentioned.
✅ Top 3 Recommended Visualizations: [('bar_chart', 0.56), ('line_chart', 0.49), ('heatmap', 0.29)]

💬 Query: What are Reddit users’ favorite fast-food chains?
💬 Response: Based on a survey in r/AskReddit, 48% of users ranked Chick-fil-A as their favorite, 30% preferred McDonald's, and 22% chose Taco Bell. Reasons included food quality, affordability, and speed.
✅ Top 3 Recommended Visualizations: [('bar_chart', 2.02), ('chord_diagram', 0.62), ('line_chart', 0.58)]

💬 Query: What are the latest updates on the Ukraine-Russia conflict?
💬 Response: Recent reports from r/worldnews indicate increased military activity along the eastern border. A UN spokesperson confirmed ongoing peace talks, but tensions remain high.
✅ Top 3 Recommended Visualizat